## <font color='gold'/>**Install and Import**

In [ ]:
!pip install \
    --requirement ../input/cellpose-wheels/requirements.txt \
    --no-index \
    --find-links ../input/cellpose-wheels/wheels

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import imageio
import matplotlib.pyplot as plt
from cellpose import models
from tqdm import tqdm
%matplotlib inline

## <font color='gold'/>**Utility functions**

In [ ]:
# Run Length Encode an instance
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
# Init model
cp_model_path = '../input/sartorius-weight/cellpose_250622_concat_epoch_500'
sz_model_path = '../input/sartorius-weight/cellpose_250622_concat_epoch_500_size.npy'

model = models.CellposeModel(gpu=True, pretrained_model=cp_model_path, concatenation=True)
sz_model = models.SizeModel(cp_model=model, pretrained_size=sz_model_path)

# Model configuration
channels = [0, 0]  # segment grayscale image

In [ ]:
test_dir = glob.glob('../input/sartorius-cell-instance-segmentation/test/*.png')
image_names, pred_annots = [], []

for file_name in tqdm(test_dir):
    img = imageio.imread(file_name)
    img_name = file_name.split('/')[-1].split('.')[0]
    predicted_diam, _ = sz_model.eval(img, channels=channels)
    masks, flows, _ = model.eval(img, diameter=predicted_diam, channels=channels, augment=True)
    
    for i in range(1, masks.max() + 1):
        pred_annot = rle_encode(masks == i)
        image_names.append(img_name)
        pred_annots.append(pred_annot)

In [ ]:
# Display result
from cellpose import plot

fig = plt.figure(figsize=(24, 8))
plot.show_segmentation(fig, img, masks, flows[0], channels=channels)
plt.tight_layout()
plt.show()

In [ ]:
pd.DataFrame({'id': image_names, 'predicted': pred_annots}).sort_values(['id']).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head(20)